In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('/Users/ajibolaoluwatobiloba/Desktop/AMOD5410- Big Data/Assignments/Assignment 3 --- ML & Spark/sleep.tsv',delimiter='\t')
trimmed_data=data.head(10000)



X = trimmed_data.drop('target', axis=1)
y = trimmed_data['target']


from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


svm = SVC()
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7485


In [2]:
# SPARKKKK

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext 
from pyspark.sql import SQLContext
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_breast_cancer
from pyspark.sql import SparkSession




data=pd.read_csv('/Users/ajibolaoluwatobiloba/Desktop/AMOD5410- Big Data/Assignments/Assignment 3 --- ML & Spark/sleep.tsv',delimiter='\t')



spark = SparkSession.builder.getOrCreate()

df_data = spark.createDataFrame(data)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 15:36:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize SparkSession
spark = SparkSession.builder.appName("BinaryClassification").getOrCreate()

# Load the data into a DataFrame
df = spark.read.csv('/Users/ajibolaoluwatobiloba/Desktop/AMOD5410- Big Data/Assignments/Assignment 3 --- ML & Spark/sleep.tsv', header=True, inferSchema=True, sep='\t')

# Assuming our target variable is named 'target' and we want to keep classes 0 and 1 only
df_filtered = df.filter((df.target == 0) | (df.target == 1))



features = [f'V{i}' for i in range(13)]  # V0 to V12

# Initializing the VectorAssembler with the feature columns
va = VectorAssembler(inputCols=features, outputCol='features')

# Transforming the DataFrame to include a features column
va_df = va.transform(df_filtered)

# Select only the features and target for modeling
va_df = va_df.select(['features', 'target'])

# Splitting the data into training and test sets
(train, test) = va_df.randomSplit([0.8, 0.2])

# Initializing the LinearSVC model
lsvc = LinearSVC(labelCol="target", maxIter=50)

# Fitting the model on the training data
lsvc_model = lsvc.fit(train)

# Making predictions on the test data
pred = lsvc_model.transform(test)

# Evaluating the model
evaluator = MulticlassClassificationEvaluator(labelCol="target", metricName="accuracy")
acc = evaluator.evaluate(pred)

# Printing the prediction accuracy
print("Prediction Accuracy: ", acc)


24/03/14 15:37:19 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Prediction Accuracy:  0.7603058510638298
